# Analysis of Treasury Bill Pricing and Interest Rate Risk

## Background
Treasury bills, otherwise known as T-Bills, are financial instruments with short-term due dates ranging from a few days to 52 weeks. They are fixed-income investments with zero coupons, meaning no coupon payments are made during the term. Instead, the price of treasury bills is calculated so that the bill receiver gets the face (par) value ($V_{P}$) at the end of the term. 

<div>
    <center>
        <img src="figs/Fig-ZC-Schematic.png" width="440"/>
    </center>
</div>

### Pricing
A zero-coupon T-bill with an annual effective market interest rate $\bar{r}$, which is specified at the time of purchase, and a term of T years has a _fair price_ of:

$$
V_{B} = \mathcal{D}_{T,0}^{-1}\cdot{V_{P}}
$$

where $\mathcal{D}_{T}$ denotes the discount factor from the time of the auction to the term of T-bill, i.e., $0\rightarrow{T}$. Typically, a constant averaged rate of interest is used to compute the discount factor:

$$
\mathcal{D}_{T,0} = (1+\bar{r})^{T}
$$

However, dynamic formulations of the discount factor with period-specific interest rates can also be used:

$$
\mathcal{D}_{T,0} = \left[\prod_{j=0}^{T-1}\left(1+r_{j+1,j}\right)\right]
$$

where $r_{j+1,j}$ terms are called short rates, i.e., the interest rate earned between period $j\rightarrow{j+1}$. Treasury bills are released for specific periods $T=$ 4, 8, 13, 26, and 52 weeks. The fair price $V_{B}$ of the T-bill is the future face value $V_{P}$ that is discounted to today’s value by the effective market interest rate $\bar{r}$, which is assumed constant over the lifetime of the treasury bill.

## Project
You purchased a 52-week T-bill with a par value $V_{P}=100.0$ USD and an effective market rate of interest $\bar{r}$ (annualized) on January 03, 2022. However, some time after purchasing the T-bill, the interest rate is now $\bar{r}^{\prime}$. Should you `sell` the T-bill before it matures or `hold` the T-bill to term?

### Objectives
 1. Load, and analyze historical T-bill interest rate data sets from 2019 to 2023 computed daily by [Federal Reserve Bank of New York](https://www.newyorkfed.org). Store these data in a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries). Select the year`"2022"` for analysis. 
 1. Develop a strategy to identify trade signals for your T-bill.
     * Test your strategy as you move forward and backward in time between January 03, 2022 and December, 31, 2023.

## Setup
In the following code block we setup the computational aspects of the problem by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this excercise and custom types to model the components of our problem.

### Packages
`Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.
* [Plots.jl](https://docs.juliaplots.org/stable/) is a plotting library in [Julia](https://julialang.org).

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/) into the notebook:

##### `loadratesfile(year::String = "2022") -> DataFrame`

> This function takes a [String](https://docs.julialang.org/en/v1/manual/strings/) encoding the year in the `YYYY` format, e.g., `"2019"` or `"2020"` and returns a [DataFrame](https://dataframes.juliadata.org/stable/) holding the daily interest rates for US Treasury Bills for various durations found on [Treasury.gov](https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_bill_rates). This project has data from `"2019"`, `"2020"`, `"2021"`, `"2022"` and `"2022"`. The year parameter has a default value of `"2022"`.

##### `build(model::Type{MyUSTreasuryBillModel}, data::NamedTuple) -> MyUSTreasuryBillModel`

> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the par value, the rate and the duration of the T-bill) and returns an instance of the `MyUSTreasuryBillModel` custom type.

##### `build(model::Type{MySymmetricBinaryLatticeModel}, data::NamedTuple)::MySymmetricBinaryLatticeModel` and `build(model::Type{MyBinaryLatticeNodeModel}, data::NamedTuple)::MyBinaryLatticeNodeModel` 

> These factory functions take information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument and construct a node or the binomial short-rate lattice model, respectively. 

##### `price(model::MyUSTreasuryBillModel, compounding::T) -> MyUSTreasuryBillModel where T <: AbstractCompoundingModel`

> This function takes a `MyUSTreasuryBillModel` instance, and a compounding model and returns an updated `MyUSTreasuryBillModel` instance that holds the price $V_{B}$ value in the `price` field of the `MyUSTreasuryBillModel` instance.

##### `𝔼(model::MySymmetricBinaryLatticeModel; level::Int = 0) -> Float64` and `𝕍(model::MySymmetricBinaryLatticeModel; level::Int = 0) -> Float64`

> These functions take a`MySymmetricBinaryLatticeModel` instance, and the level and the tree (`0`-based) and compute the [expectation](https://en.wikipedia.org/wiki/Expected_value) and [variance](https://en.wikipedia.org/wiki/Variance) of the short-rates for the given level (time-step) of the lattice. 

### Types
`Include.jl` loads some [problem-specific types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of T-bill pricing. 

* `MyUSTreasuryBillModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$ and the interest rate $\bar{r}$ for a zero-coupon treasury bill. You construct a `MyUSTreasuryBillModel` instance using the `build` method described above.
* `DiscreteCompoundingModel` and `ContinuousCompoundingModel` are [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that let our code know which compounding model we wish to use.
* The `MyBinaryLatticeNodeModel` and `MySymmetricBinaryLatticeModel` mutable types hold information about the nodes, and the binomial lattice, respectively, for the short-rate calculations. 

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-1`


## Set constant values 
We will need several constants, regardless of the year we choose to analyze. First, we will always analyze 52-week data, so we encode this choice in the `key` variable. Next, we'll assume a discrete compounding model; thus, we instantiate an instance of the `DiscreteCompoundingModel` type and then store this instance in the `compounding` variable. Lastly, the T-bill’s face (par) value will always be $V_{P}$ = 100.0 USD.

In [2]:
number_of_weeks_per_year = 52;
key = Symbol("52 WEEKS COUPON EQUIVALENT");
Vₚ = 100.0;
T = (52.0/number_of_weeks_per_year);
compounding = DiscreteCompoundingModel();

## Objective 1: Load historical interest rate dataset
To begin, we gather interest rate data from `2019` to `2022`, using rates from recently auctioned bills collected by the Federal Reserve Bank of New York on every business day.  For this analysis, we use the `52 WEEKS COUPON EQUIVALENT` data as a proxy for the effective market interest rate $\bar{r}$ on each trading day for a 52-week T-bill.

### Implementation
We store the rate information in a `rate_data_dictionary` variable, which is a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) with keys representing years (from the `years` array, type `String`) and values as a `DataFrame` holding rate values. We load each year by iterating through the `years` array and passing the corresponding `year` parameter to the `loadratesfile(...)` function:

In [19]:
rate_data_dictionary = Dict{String,DataFrame}();
years = ["2019","2020","2021","2022"];
for year ∈ years
    rate_data_dictionary[year] = loadratesfile(year=year);
end

Now that have loaded the rates dataset, let's analyze `2022`, and save our selection in the variable `selected_year`. Then access the corresponding rate values `DataFrame` for the `selected_year` from the `rate_data_dictionary`. Store the rate data in the `dataset` variable:

In [4]:
selected_year = "2022"
dataset = rate_data_dictionary[selected_year];

## Objective 2: Identify possible treasury bill trade signals
Deciding whether to `buy`, `sell`, or `hold` a T-bill, or any other security, depends on your investment objectives. Treasury bills are generally considered secure, short-term investments, but their prices can fluctuate depending on the interest rate. If the price of a T-bill changes, you could `sell` it before maturity and earn an immediate capital gain or loss. However, it's crucial to know when this makes sense.

To start answering this question, let's examine our historical data set to identify potential trade signals, which indicate changes in the T-bill's price that could result in a capital gain or loss. We'll use the [log return](https://en.wikipedia.org/wiki/Rate_of_return) $R_{ij}$ between two dates ($i\geq{j}$) to measure the potential gain or loss:

$$
R_{i,j} = 100\cdot{\ln\left(\frac{V_{B,i}}{V_{B,j}}\right)}
$$

Here, $V_{B,k}$ refers to the price of a 52-week T-bill on day $k$ in the dataset, and $\ln$ denotes the [natural logarithm](https://en.wikipedia.org/wiki/Natural_logarithm). If you have two possible actions $a\in\left\{\text{sell},\text{hold}\right\}$ you can propose the following policy: Implement a `sell` action if $R_{ij}>\alpha$, where $\alpha>0$ is a user-specified trade threshold. Otherwise, implement a `hold` action.

### Prerequisite: Compute the price of a 52-week T-bill for each trading day of 2022
To calculate the cost of a 52-week T-bill for each trading day in 2022, start by finding out how many trading days are in `2022` using the `nrow(…)` function. Save this number in the `number_of_trading_days` variable. Then create a one-dimensional `models` array containing instances of `MyUSTreasuryBillModel`. Finally, using a `for` loop, construct `MyUSTreasuryBillModel` instances,  and compute the price, for a 52-week treasury bill using the effective market rate of interest $\bar{r}$ (found in the `dataset` variable) for each trading day. Store these instances in the `models` array.

__Reminder__: the `dataset` is organized from the newest (Dec-31-2022) to oldest (Jan-03-2022).

In [5]:
number_of_trading_days = nrow(dataset)
models = Array{MyUSTreasuryBillModel,1}(undef, number_of_trading_days);
for i ∈ 1:number_of_trading_days
    
    r̄ = dataset[i,key]*(1/100)
    model = build(MyUSTreasuryBillModel, (
        par = Vₚ, T = T, rate=r̄)) |> compounding
    
    models[i] = model;
end

### Compute the return $R_{ij}$ array for all combinations of $(i,j)$:

In [41]:
reversed_models_array = reverse(models);
log_return_array = Array{Float64,2}(undef, number_of_trading_days, number_of_trading_days)
fill!(forward_return_array,0.0);
for i ∈ 1:number_of_trading_days
    
    purchased_model = reversed_models_array[i];
    purchase_price = purchased_model.price
    
    for j ∈ 1:number_of_trading_days
        next_model = reversed_models_array[j];
        next_price = next_model.price
        log_return_array[i,j] = log(next_price/purchase_price)*100.0;
    end
end

In [42]:
log_return_array

249×249 Matrix{Float64}:
  0.0         0.0199243  -0.00996066  -0.0398367   …  -3.99331     -4.02202
 -0.0199243   0.0        -0.0298849   -0.059761       -4.01323     -4.04194
  0.00996066  0.0298849   0.0         -0.029876       -3.98335     -4.01206
  0.0398367   0.059761    0.029876     0.0            -3.95347     -3.98218
  0.0199203   0.0398446   0.00995966  -0.0199164      -3.97339     -4.0021
  0.0497934   0.0697176   0.0398327    0.00995669  …  -3.94351     -3.97222
  0.0497934   0.0697176   0.0398327    0.00995669     -3.94351     -3.97222
  0.0597491   0.0796733   0.0497884    0.0199124      -3.93356     -3.96227
  0.0597491   0.0796733   0.0497884    0.0199124      -3.93356     -3.96227
  0.0896102   0.109534    0.0796495    0.0497735      -3.9037      -3.93241
  0.159252    0.179176    0.149291     0.119415    …  -3.83405     -3.86276
  0.139359    0.159283    0.129398     0.0995223      -3.85395     -3.88266
  0.169196    0.189121    0.159236     0.12936        -3.82411  